# Midterm

In [53]:
import numpy as np
import pandas as pd
import matplotlib as mp
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS 
from statsmodels.sandbox.regression.gmm import GMM
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,recall_score, precision_score, f1_score
import warnings

## Part 1

import the database required for part 1

In [54]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 2\Predictive Analytics\Midterm\midterm_partone.csv")

get an overview of the dataset

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696 entries, 0 to 1695
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Constant            1696 non-null   int64  
 1   Stock Change        1696 non-null   float64
 2   Inventory Turnover  1696 non-null   float64
 3   Operating Profit    1696 non-null   float64
 4   Interaction Effect  1696 non-null   float64
 5   Current Ratio       1696 non-null   float64
 6   Quick Ratio         1696 non-null   float64
 7   Debt Asset Ratio    1696 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 106.1 KB


In [56]:
df.head()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446


Run the Linear Regression and get the predicted values

In [57]:
model_iv = sm.OLS(df["Inventory Turnover"],df[["Constant","Current Ratio","Quick Ratio","Debt Asset Ratio"]]).fit()
endog_predict = model_iv.predict(df[["Constant","Current Ratio","Quick Ratio","Debt Asset Ratio"]])
df["Endogenous Param"] = endog_predict

Run a second step linear regression

In [58]:
model_2sls = sm.OLS(df["Stock Change"], df[["Constant","Endogenous Param","Operating Profit","Interaction Effect"]]).fit()
model_2sls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Stock Change   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     8.530
Date:                Sun, 12 Nov 2023   Prob (F-statistic):           1.27e-05
Time:                        20:52:07   Log-Likelihood:                -1186.5
No. Observations:                1696   AIC:                             2381.
Df Residuals:                    1692   BIC:                             2403.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Constant              -0.0176      0.020     -0.896      0.370      -0.056       0.021
Endogenous Param       0.0011      0.001      1.827      0.068   -7.76e-05       0.002
Operating Profit      -0.1201      0.028     -4.319      0.000      -0.175      -0.066
Interaction Effect     0.0014      0.000      3.621      0.000       0.001       0.002
==============================================================================
Omnibus:                      368.832   Durbin-Watson:                   2.243
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3433.920
Skew:                           0.742   Prob(JB):                         0.00
Kurtosis:                       9.811   Cond. No.                         109.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-squared and adj. R-squared: This model explains only a small portion of the Stock Change variability. This is due to the low values of 0.15 and 0.13.

F-statistic: The model has a high F-statistic of 8.530 which supports the model being statistically significant

p-value: the high f-statistic and low p-value indicates at least one of the predictor variables used is significantly related to the Stock Change. 

t-statistic: tells us how significant each of the explanantory variables are, the farther from 0 the more significant. None of the variables are very close to 0 but Endogeneous Param is the closest at 1.827.

p-value: tells us the probaility that coeficients are not significantly different from 0.

Both Operating profit and interaction effect have p values of 0 which shows they are significant. Endogeneous Param has a p value greater than 0.05 which shows it is not statistically significant. 

GMM model with no changes

In [59]:
y_vals  = np.array(df["Stock Change"])
x_vals  = np.array(df[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(df[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] 
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] 
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] 
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] 

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1])
res = gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=4).fit(beta0)

res.summary()

Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.6317
Model:                            gmm   Prob (Hansen J):                 0.729
Method:                           GMM                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        20:52:07                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0200      0.021     -0.964      0.335      -0.061       0.021
p 1            0.0011      0.001      1.843      0.065   -6.89e-05       0.002
p 2           -0.1071      0.032     -3.370      0.001      -0.169      -0.045
p 3            0.0011      0.000      2.760      0.006       0.000       0.002
==============================================================================
"""

Coefficients:

Constant: coefficient of -0.0200 and a p-value of 0.335

Endogenous Param: coefficient of 0.0011 and a p-value of 0.065

Operating Profit: coefficient of -0.1071 and a p-value of 0.001

Interaction Effect: coefficient of 0.0011 and a p-value of 0.006

GMM model after subtracting the delta value

In [60]:
class gmm_with_delta(GMM):
    def momcond(self, params):
        p0, p1, p2, p3, delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument 

        error0 =  endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] 
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = ((endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0]) - delta
        error4 = ((endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1]) - delta
        error5 = ((endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2]) - delta

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g

y_vals = np.array(df["Stock Change"])
x_vals = np.array(df[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])
iv_vals = np.array(df[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])

# Fit the GMM model with the δ term
beta0_with_delta = np.array([0.1, 0.1, 0.1, 0.1, 0.1])  # Initial guess for parameters including delta
res_with_delta = gmm_with_delta(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=5).fit(beta0_with_delta)
res_with_delta.summary()

Optimization terminated successfully.
         Current function value: 0.000031
         Iterations: 10
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.000345
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 7
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            gmm_with_delta Results                            
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.5862
Model:                 gmm_with_delta   Prob (Hansen J):                 0.444
Method:                           GMM                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        20:52:07                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0208      0.021     -0.986      0.324      -0.062       0.020
p 1            0.0011      0.001      1.839      0.066   -7.31e-05       0.002
p 2           -0.1062      0.032     -3.316      0.001      -0.169      -0.043
p 3            0.0011      0.000      2.688      0.007       0.000       0.002
p 4           -0.0006      0.003     -0.213      0.831      -0.006       0.005
==============================================================================
"""

Coefficients:

Constant: coefficient of -0.0208 and a p-value of 0.324

Endogenous Param: coefficient of 0.0011 and a p-value of 0.066

Operating Profit: coefficient of -0.1062 and a p-value of 0.001

Interaction Effect: coefficient of 0.0011 and a p-value of 0.007

Delta Value: coefficient of -0.0006 and a p-value of 0.831

### Summary

The coefficients for the constant, the endogeneous param, and the delta value are not statistically significant. the p-value for operating profit and interaction effect is less than 0.05 which shows it is statistically significant. 

Within the context of the GMM model that includes the delta value, there is no strong evidence that the industry experts claim can be true. The δ value does not have a significant effect on the model.

## Part 2

### Question 1

Import database required for question 2

In [61]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 2\Predictive Analytics\Midterm\midterm_parttwo.csv")

Overview of the data

In [62]:
df.head()

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8081 entries, 0 to 8080
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Years of Education after High School  8081 non-null   int64 
 1   Requested Credit Amount               8081 non-null   object
 2   Number of Dependents                  8081 non-null   object
 3   Monthly Income                        8081 non-null   object
 4   Monthly Expense                       8081 non-null   object
 5   Marital Status                        8081 non-null   object
 6   Credit Rating                         8081 non-null   object
dtypes: int64(1), object(6)
memory usage: 442.1+ KB


get dummies for categorical variables

In [64]:
df = pd.get_dummies(df, columns=['Requested Credit Amount', 'Marital Status', 'Number of Dependents','Monthly Income', 'Monthly Expense'], drop_first=True, dtype=int)

define x and y columns

In [65]:
y = df['Credit Rating']
X = df.drop('Credit Rating', axis=1)

split the dataset into testing and training, 50/50

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

Run the basic Logistic Regression Model

In [67]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

create the prediction values

In [68]:
y_pred = model.predict(X_test)

Retrieve all of the required measures

In [69]:
conf_matrix = confusion_matrix(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label='Positive')
precision = precision_score(y_test, y_pred, pos_label='Positive')
f1 = f1_score(y_test, y_pred, pos_label='Positive')

In [70]:
print("Confusion Matrix:\n", conf_matrix)
print("\nRecall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)

Confusion Matrix:
 [[   0  577]
 [   0 3464]]

Recall: 1.0
Precision: 0.8572135609997525
F1 Score: 0.9231179213857428


Recall: a recall value of 1 shows the model identified all positive instances while going through the test set

Precision: about 86% of the values predicted as positive were truly positive

F1 Score: a strong f1-score of 0.92

### Question 2

set the approval threshold to 0.15

In [71]:
apps_granted = 0.15

In [72]:
y_probs = model.predict_proba(X_test)[:, 1]

In [73]:
threshold = np.percentile(y_probs, 100 - (apps_granted * 100))

In [74]:
y_pred_updated = [1 if p >= threshold else 0 for p in y_probs]

In [75]:
y_pred_updated_map = ['Negative' if pred == 0 else 'Positive' for pred in y_pred_updated]

Retrieve all of the required measures

In [76]:
conf_matrix = confusion_matrix(y_test, y_pred_updated_map)
recall = recall_score(y_test, y_pred_updated_map, pos_label='Positive')
precision = precision_score(y_test, y_pred_updated_map, pos_label='Positive')
f1 = f1_score(y_test, y_pred_updated_map, pos_label='Positive')

In [77]:
print("Confusion Matrix:\n", conf_matrix)
print("\nRecall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)

Confusion Matrix:
 [[ 495   82]
 [2936  528]]

Recall: 0.15242494226327943
Precision: 0.8655737704918033
F1 Score: 0.2592047128129602


Recall: a recall value of 0.15 shows the model identified 15% of positive instances while going through the test set

Precision: about 87% of the values predicted as positive were truly positive

F1 Score: a much lower f1-score of 0.26